In [ ]:
!pip install kaggle

# **Data Acquisition**

Dataset acquisition from Kaggle.

In [ ]:
import pandas as pd
import os

In [ ]:
os.environ["KAGGLE_CONFIG_DIR"] = '/content'

In [ ]:
!kaggle datasets download -d dbdmobile/myanimelist-dataset

Dataset URL: https://www.kaggle.com/datasets/dbdmobile/myanimelist-dataset
License(s): DbCL-1.0
100% 1.79G/1.80G [00:22<00:00, 58.6MB/s]
100% 1.80G/1.80G [00:22<00:00, 86.1MB/s]


In [ ]:
!unzip \*.zip && rm *.zip

Archive:  myanimelist-dataset.zip
  inflating: anime-dataset-2023.csv  
  inflating: anime-filtered.csv      
  inflating: final_animedataset.csv  
  inflating: user-filtered.csv       
  inflating: users-details-2023.csv  
  inflating: users-score-2023.csv    


In [ ]:
anime_df = pd.read_csv('/content/anime-dataset-2023.csv')

In [ ]:
anime_df

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,无脑魔女,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",...,UNKNOWN,Web manga,Unknown,PG-13 - Teens 13 or older,UNKNOWN,24723,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1386/...
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1383/...
24902,55733,Di Yi Xulie,The First Order,第一序列,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1130/...
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",...,UNKNOWN,Original,3 min,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1931/...


In [ ]:
anime_df.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

Replacing all the unknown values with *None* values

In [ ]:
anime_df.replace(to_replace=['Unknown','UNKNOWN', 'No description available for this anime.'], value = None, inplace = True)

In [ ]:
anime_df.isnull().sum()

anime_id            0
Name                0
English name    14577
Other name        128
Score            9213
Genres           4929
Synopsis         4535
Type               74
Episodes          611
Aired               0
Premiered       19399
Status              0
Producers       13350
Licensors       20170
Studios         10526
Source           3689
Duration          663
Rating            669
Rank             4612
Popularity          0
Favorites           0
Scored By        9213
Members             0
Image URL           0
dtype: int64

In [ ]:
anime_df = anime_df[['Name', 'Genres', 'Synopsis', 'Producers', 'Studios', 'Image URL']]

In [ ]:
anime_df[anime_df['Name'] == 'Dragon Ball Super']

,Name,Genres,Synopsis,Producers,Studios,Image URL
10577,Dragon Ball Super,"Action, Adventure, Comedy, Fantasy","Seven years after the events of Dragon Ball Z,...","Yomiko Advertising, Fuji TV",Toei Animation,https://cdn.myanimelist.net/images/anime/7/746...


Considering only the anime with a synopsis.

In [ ]:
anime_df = anime_df[anime_df['Synopsis'].notnull()]

# **Data Cleaning**

Some anime in the dataset are just 3/4 minutes music video. We proceed to remove these observations since they are not in accordance with the purpose of the project.

In [ ]:
anime_df = anime_df[~anime_df['Synopsis'].str.contains(r'[Mm]usic video', regex = True)].reset_index(drop = True)

In [ ]:
anime_df.head()

,Name,Genres,Synopsis,Producers,Studios,Image URL
0,Cowboy Bebop,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",Bandai Visual,Sunrise,https://cdn.myanimelist.net/images/anime/4/196...
1,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi","Another day, another bounty—such is the life o...","Sunrise, Bandai Visual",Bones,https://cdn.myanimelist.net/images/anime/1439/...
2,Trigun,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",Victor Entertainment,Madhouse,https://cdn.myanimelist.net/images/anime/7/203...
3,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,"Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,https://cdn.myanimelist.net/images/anime/10/19...
4,Bouken Ou Beet,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,"TV Tokyo, Dentsu",Toei Animation,https://cdn.myanimelist.net/images/anime/7/215...


Consider only those anime whose synopsis contains more than 50 words, in such a way the subsequent analysis can be more accurate.

In [ ]:
anime_df['length_synopsis'] = anime_df['Synopsis'].apply(lambda x: len(x.split()))

In [ ]:
anime_df = anime_df[anime_df['length_synopsis'] >= 50].reset_index(drop = True)

In [ ]:
anime_df = anime_df.drop('length_synopsis', axis = 1)

In [ ]:
anime_df

,Name,Genres,Synopsis,Producers,Studios,Image URL
0,Cowboy Bebop,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",Bandai Visual,Sunrise,https://cdn.myanimelist.net/images/anime/4/196...
1,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi","Another day, another bounty—such is the life o...","Sunrise, Bandai Visual",Bones,https://cdn.myanimelist.net/images/anime/1439/...
2,Trigun,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",Victor Entertainment,Madhouse,https://cdn.myanimelist.net/images/anime/7/203...
3,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,"Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,https://cdn.myanimelist.net/images/anime/10/19...
4,Bouken Ou Beet,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,"TV Tokyo, Dentsu",Toei Animation,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...
9739,The Demon Queen Has a Death Wish,"Fantasy, Romance",I wanted to die.. until I met him.\nAfter brea...,None,None,https://cdn.myanimelist.net/images/anime/1527/...
9740,No More Princes,"Fantasy, Romance",html.serialization_informationFIN.\nジャンル\nFant...,None,None,https://cdn.myanimelist.net/img/sp/icon/apple-...
9741,The Forgotten Princess Just Wants Peace,"Fantasy, Romance",I'm the daughter of a Duke?!\nWhile locked up ...,None,None,https://cdn.myanimelist.net/images/anime/1887/...
9742,Beauty and the Brawn,"Boys Love, Comedy",Haeun is a nobleman who falls hard for Dullseh...,None,None,https://cdn.myanimelist.net/images/anime/1269/...


## **Grouping similar genres**

Some genres are very similar to others (e.g. *Boys Love* and *Girls Love* are redundant genres), so we will proceed to merge similar genres into one category.
Groups are made in the following way:

* *Action and Adventure* includes the genres *Action* and *Adventure*
* *Fantasy and Supernatural* includes *Sci-Fi*, *Supernatural* and *Fantasy*
* *Drama and Mistery* includes *Drama* and *Mystery*
* *Comedy and Romance* includes *Comedy* and *Romance*
* *Sport and Slice of Life* includes *Sports* and *Slice of Life*
* *Horror and Suspense* includes *Horror* and *Suspense*
* *Gourmet and Avant Garde* includes *Gourmet* and *Avant Garde*
* *Erotica and Hentai* includes *Ecchi*, *Erotica* and *Hentai*
* *Love* includes *Girls Love* and *Boys Love*

Moreover, the category *Award Winning* has been removed.

In [ ]:
anime_df['Genres'] = anime_df['Genres'].str.replace(r'Award Winning,?\s*', '', regex=True) # Removing Award Winning genre

In [ ]:
def make_list(text):

  if not isinstance(text, float):

    text = text.replace(' ','')
    text = text.split(',')

  return text

In [ ]:
anime_df['categories'] = anime_df['Genres'].apply(lambda x: make_list(x))

In [ ]:
anime_df['categories']

0                             [Action, Sci-Fi]
1                             [Action, Sci-Fi]
2                  [Action, Adventure, Sci-Fi]
3       [Action, Drama, Mystery, Supernatural]
4           [Adventure, Fantasy, Supernatural]
                         ...                  
9739                        [Fantasy, Romance]
9740                        [Fantasy, Romance]
9741                        [Fantasy, Romance]
9742                        [BoysLove, Comedy]
9743                                [BoysLove]
Name: categories, Length: 9744, dtype: object

In [ ]:
anime_df_genrenotnull = anime_df[anime_df['Genres'].notnull()]
anime_df_genrenull = anime_df[anime_df['Genres'].isnull()]

In [ ]:
genre_mapping = {
    'Action and Adventure': {'Action', 'Adventure'},
    'Fantasy and Supernatural': {'Sci-Fi', 'Supernatural','Fantasy'},
    'Drama and Mistery': {'Drama', 'Mystery'},
    'Comedy and Romance': {'Comedy', 'Romance'},
    'Sport and Slice of Life': {'Sports', 'SliceofLife'},
    'Horror and Suspense': {'Horror', 'Suspense'},
    'Gourmet and Avant Garde': {'Gourmet', 'AvantGarde'},
    'Erotica e Hentai': {'Ecchi', 'Erotica', 'Hentai'},
    'Love': {'GirlsLove', 'BoysLove'}
}

In [ ]:
def categorize_genres(genres_list):
    categories = set()
    for genre in genres_list:
        for category, genre_set in genre_mapping.items():
            if genre in genre_set:
                categories.add(category)
                break
    return list(categories)

In [ ]:
anime_df_genrenotnull['categories'] = anime_df_genrenotnull['categories'].apply(categorize_genres)

<ipython-input-29-25bfb10e56ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df_genrenotnull['categories'] = anime_df_genrenotnull['categories'].apply(categorize_genres)


In [ ]:
anime_df_genrenotnull

,Name,Genres,Synopsis,Producers,Studios,Image URL,categories
0,Cowboy Bebop,"Action, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",Bandai Visual,Sunrise,https://cdn.myanimelist.net/images/anime/4/196...,"[Fantasy and Supernatural, Action and Adventure]"
1,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi","Another day, another bounty—such is the life o...","Sunrise, Bandai Visual",Bones,https://cdn.myanimelist.net/images/anime/1439/...,"[Fantasy and Supernatural, Action and Adventure]"
2,Trigun,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",Victor Entertainment,Madhouse,https://cdn.myanimelist.net/images/anime/7/203...,"[Fantasy and Supernatural, Action and Adventure]"
3,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,"Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,https://cdn.myanimelist.net/images/anime/10/19...,"[Drama and Mistery, Action and Adventure, Fant..."
4,Bouken Ou Beet,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,"TV Tokyo, Dentsu",Toei Animation,https://cdn.myanimelist.net/images/anime/7/215...,"[Fantasy and Supernatural, Action and Adventure]"
...,...,...,...,...,...,...,...
9739,The Demon Queen Has a Death Wish,"Fantasy, Romance",I wanted to die.. until I met him.\nAfter brea...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1527/...,"[Fantasy and Supernatural, Comedy and Romance]"
9740,No More Princes,"Fantasy, Romance",html.serialization_informationFIN.\nジャンル\nFant...,NaN,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,"[Fantasy and Supernatural, Comedy and Romance]"
9741,The Forgotten Princess Just Wants Peace,"Fantasy, Romance",I'm the daughter of a Duke?!\nWhile locked up ...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1887/...,"[Fantasy and Supernatural, Comedy and Romance]"
9742,Beauty and the Brawn,"Boys Love, Comedy",Haeun is a nobleman who falls hard for Dullseh...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1269/...,"[Love, Comedy and Romance]"


In [ ]:
anime_df_genrenotnull['Genres'] = anime_df_genrenotnull['categories']
anime_df_genrenotnull.drop('categories', inplace = True, axis = 1)

<ipython-input-31-31c35a8c1160>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df_genrenotnull['Genres'] = anime_df_genrenotnull['categories']
<ipython-input-31-31c35a8c1160>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df_genrenotnull.drop('categories', inplace = True, axis = 1)


In [ ]:
anime_df_genrenotnull

,Name,Genres,Synopsis,Producers,Studios,Image URL
0,Cowboy Bebop,"[Fantasy and Supernatural, Action and Adventure]","Crime is timeless. By the year 2071, humanity ...",Bandai Visual,Sunrise,https://cdn.myanimelist.net/images/anime/4/196...
1,Cowboy Bebop: Tengoku no Tobira,"[Fantasy and Supernatural, Action and Adventure]","Another day, another bounty—such is the life o...","Sunrise, Bandai Visual",Bones,https://cdn.myanimelist.net/images/anime/1439/...
2,Trigun,"[Fantasy and Supernatural, Action and Adventure]","Vash the Stampede is the man with a $$60,000,0...",Victor Entertainment,Madhouse,https://cdn.myanimelist.net/images/anime/7/203...
3,Witch Hunter Robin,"[Drama and Mistery, Action and Adventure, Fant...",Robin Sena is a powerful craft user drafted in...,"Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,https://cdn.myanimelist.net/images/anime/10/19...
4,Bouken Ou Beet,"[Fantasy and Supernatural, Action and Adventure]",It is the dark century and the people are suff...,"TV Tokyo, Dentsu",Toei Animation,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...
9739,The Demon Queen Has a Death Wish,"[Fantasy and Supernatural, Comedy and Romance]",I wanted to die.. until I met him.\nAfter brea...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1527/...
9740,No More Princes,"[Fantasy and Supernatural, Comedy and Romance]",html.serialization_informationFIN.\nジャンル\nFant...,NaN,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...
9741,The Forgotten Princess Just Wants Peace,"[Fantasy and Supernatural, Comedy and Romance]",I'm the daughter of a Duke?!\nWhile locked up ...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1887/...
9742,Beauty and the Brawn,"[Love, Comedy and Romance]",Haeun is a nobleman who falls hard for Dullseh...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1269/...


In [ ]:
anime_df_genrenull.drop('categories', inplace = True, axis = 1)

<ipython-input-40-09e05bb1c90f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df_genrenull.drop('categories', inplace = True, axis = 1)


In [ ]:
anime_df_concat = pd.concat([anime_df_genrenotnull, anime_df_genrenull])

In [ ]:
anime_df_concat

,Name,Genres,Synopsis,Producers,Studios,Image URL
0,Cowboy Bebop,"[Fantasy and Supernatural, Action and Adventure]","Crime is timeless. By the year 2071, humanity ...",Bandai Visual,Sunrise,https://cdn.myanimelist.net/images/anime/4/196...
1,Cowboy Bebop: Tengoku no Tobira,"[Fantasy and Supernatural, Action and Adventure]","Another day, another bounty—such is the life o...","Sunrise, Bandai Visual",Bones,https://cdn.myanimelist.net/images/anime/1439/...
2,Trigun,"[Fantasy and Supernatural, Action and Adventure]","Vash the Stampede is the man with a $$60,000,0...",Victor Entertainment,Madhouse,https://cdn.myanimelist.net/images/anime/7/203...
3,Witch Hunter Robin,"[Drama and Mistery, Action and Adventure, Fant...",Robin Sena is a powerful craft user drafted in...,"Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,https://cdn.myanimelist.net/images/anime/10/19...
4,Bouken Ou Beet,"[Fantasy and Supernatural, Action and Adventure]",It is the dark century and the people are suff...,"TV Tokyo, Dentsu",Toei Animation,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...
9700,Mecha Mecha Karisuma,NaN,"This is the Charisma House. Today, too, the ch...",NaN,NaN,https://cdn.myanimelist.net/images/anime/1831/...
9707,Follow Your Fantasy,NaN,Witness stunning animation featuring Cygames a...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1003/...
9708,Pon no Michi,NaN,"Nashiko Jippensha, a high school girl living i...","Bit grooove promotion, SUPA LOVE",OLM,https://cdn.myanimelist.net/images/anime/1796/...
9709,A Flash | Honkai: Star Rail,NaN,"Year 7380 Star Calendar, Jingliu, the mara-str...",NaN,NaN,https://cdn.myanimelist.net/images/anime/1547/...


In [ ]:
anime_df_concat.reset_index(inplace = True, drop = True)

# **Saving the dataframe**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/Text Mining Project/'

In [ ]:
with open(path+'animelist_categorized.pkl', 'wb') as f:
    anime_df_concat.to_pickle(f)